In [1]:
%load_ext autoreload 

%autoreload 2
import torch
import torch.nn as nn
import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm
from XAI_methods import GradCAM,LayerCAM,ScoreCAM,GradCAMPlusPlus,XGradCAM,FinerCAM
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from XAI_methods.XAI_utils import show_cam_on_image
import matplotlib.pyplot as plt